In [21]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import time

#chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

# try:
#     driver = webdriver.Chrome(f'91/chromedriver')   
# except:
#     chromedriver_autoinstaller.install(True)
#     driver = webdriver.Chrome(f'91/chromedriver')

driver = webdriver.Chrome(f'../chromedriver')



In [29]:
driver.get("https://v4.map.naver.com")
driver.find_elements_by_css_selector("button.btn_close")[1].click()
# spmc_zoomout
driver.implicitly_wait(10)
#         a:nth-child(3)#

search_box = driver.find_element_by_css_selector("input#search-input")
search_box.send_keys("판교")

search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)
search_box.clear()

#지도레벨 축소
# for i in range(3):    
#     driver.find_elements_by_css_selector("a.spmc_zoomout")[0].click()
#     time.sleep(0.25)


#print(chrome_ver)

## store ids 
store_ids = [];

In [23]:
store_id_path = 'store_id.txt'

def get_store_id(webl_obj) :
    for store in webl_obj:
        str_id = store.get_attribute("class") + ""
        store_ids.append(str_id.split(',')[1]);
        print(str_id.split(',')[1])
        #WriteTxtFiles(store_id_path, str_id.split(',')[1])


def WriteTxtFiles(path, store_id):    
    with open(path, "a") as file:
        file.write(store_id + ",")
        file.close()



In [30]:
search_query  = ['식당']
search_box = driver.find_element_by_css_selector("input#search-input")
search_box.clear()
current_page_num = 1;
max_search_page_num = 50;

search_box.send_keys(search_query)
#         a:nth-child(3)#
search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)

# 우선 store_id는 100개만 수집
for i in range(max_search_page_num):
    next_button = driver.find_element_by_css_selector("div.paginate").find_elements_by_tag_name("a")    
    stores = driver.find_elements_by_css_selector("ul.lst_site>li")
    get_store_id(stores)
    current_page_num += 1;
    
    for nextItem in next_button:
        if(nextItem.text == "다음"):
            nextItem.click()
            time.sleep(1)
            break
            
        if(nextItem.text == str(current_page_num)):
            nextItem.click()
            time.sleep(1)
            break

store_id_set = set(store_ids)
for store_id in store_id_set:
    WriteTxtFiles(store_id_path, store_id)

36609692
1978442785
1627952506
36620164
38668100
20747211
1017006923
21889352
21553337
35637628
843434158
36614869
37855133
1238642457
463176428
1681273162
37243953
1187221669
37869908
37112553
34427989
1972890168
1795337574
35244004
1948518379
31509929
37928792
31487795
20779486
1642068776
43668661
38338661
20819236
36426428
1983640525
654793321
37792366
35103362
1349600448
1160907249
38404209
36305625
1807798408
1031190455
20997301
1967151001
34754706
634059689
1800965862
36974071
21149144
1410300251
35905799
37827976
37362330
1014550775
21791082
31053361
1570163755
37803338
36112281
1867012609
35228220
35701682
1154174040
38737658
45891145
1341714515
34582866
1268902537
32594068
36304933
1800463318
1755630989
1986722486
1054721975
36939453
32473186
36513079
38419329
31125350
1690982169
38312372
1083405900
1746231153
1192950182
20573606
38325627
1967020590
38705391
1947264225
1870030675
1467346720
31016348
1389360249
1593577547
1238485493
35195126
1535187802
32981857
13592478
3789566

In [31]:
# 수집된 store_id로 방문자 리뷰 데이터를 수집한다
from bs4 import BeautifulSoup
import requests

with open("store_id.txt", "r") as file:
    store_ids = file.read()
    file.close()
    
driver = webdriver.Chrome(f'../chromedriver')

for store_id in store_ids.split(","):
    if(store_id):
        print("================")
        
        url = f'https://pcmap.place.naver.com/restaurant/{store_id}/review/visitor'
        htm = driver.get(url)
        time.sleep(1)
        
        try:
            for i in range(20):
                more_btn = driver.find_element_by_class_name("_3iTUo")
                if(more_btn):
                    more_btn.click()
                    time.sleep(0.5)
                else:
                    break;
        except Exception as ex:
            print("error catch : " , ex)
        
        print("id : " + store_id)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # li _2Cv-r -> 부모 컨테이너 // span _2tObC  ->별점 // span WoYOw ->리뷰 내용

        for ll_item in soup.find_all('li',{'class' : '_2Cv-r'}):

            rating_ll = ll_item.find('span',{'class' : '_2tObC'})
            review_ll = ll_item.find('span',{'class' : 'WoYOw'})

            if(review_ll):
                #10자 이상 리뷰만 의미있는 값으로 본다.
                if(len(review_ll.text)>10):
                    #3점 ~ 4점대 리뷰는 거른다 (3점미만 : 부정 , 5점 : 긍정)
                    if(float(rating_ll.text) < 3 or float(rating_ll.text) > 4.5):
                        review_content = review_ll.text
                        label = "0" #0 : 긍정, 1 부정
                        if("\n" in review_ll.text):
                            review_content = review_ll.text.replace("\n"," ")
                            
                        #3점 미만이면 부정평가
                        if(float(rating_ll.text) < 3 ):
                            label = "1";
                            
                            with open("review_data.txt", "a") as file:
                                # id , 별점, 리뷰내용, 레이블(0:긍정, 1:부정)
                                file.write(store_id+"\t" + rating_ll.text+"\t" + review_content +"\t"+ label+ "\n") 
    #                     print("rating : " + rating_ll.text)
    #                     print("review : " + review_ll.text)
        file.close()
#        driver.close()
        

id : 37855133
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1387099270
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1112803147
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 35905799
id : 38591456
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1701737404
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 31509929
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session i

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1883977562
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1870030675
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1153831976
id : 38695874
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1008432874
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 31016348
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1151481039
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1111370028
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1054464736
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1654443384
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1867012609
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.10

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1728739598
id : 1972890168
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 36417179
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 35103362
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1800813298
id : 34573564
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1154174040
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1755630989
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 38338661
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 33142227
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 942176013
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 526337679
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 36267003
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 37933098
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1593577547
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 31125350
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 36543254
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id :

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 36668197
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1947264225
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 37803338
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1086756066
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 38737658
id : 20779486
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0